# **Step2 제공파일_AI면접관 Agent_with Gradio**

## **1. 환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project_genai)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 라이브러리

In [2]:
!pip install langchain_openai langchain_core langchain-community -q
!pip install -U langgraph
!pip install PyMuPDF python-docx gradio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.
   ━━━━━━━━

### (3) OpenAI API Key 확인
* api_key.txt 파일에 다음의 키를 등록하세요.
    * OPENAI_API_KEY

In [3]:
import os

def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/project_genai/'
# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

In [ ]:
print(os.environ['OPENAI_API_KEY'][:30])

## **2. App.py**

* 아래 코드에, Step1 혹은 고도화 된 Step2 파일 코드를 붙인다.
    * 라이브러리
    * 함수들과 그래프
* Gradio 코드는 그대로 사용하거나 일부 수정 가능

%%writefile app.py  

밑에 나오는 코드를 app.py에 저장한다는 의미

In [16]:
%%writefile app.py

####### 1. 라이브러리 로딩 #######
import pandas as pd
import numpy as np
import os
import openai
import random
import ast
import fitz  # PyMuPDF
from docx import Document
import json
import warnings
from pydantic import BaseModel

# Gradio
import gradio as gr
import tempfile
import time

warnings.filterwarnings("ignore", category=DeprecationWarning)

from typing import Annotated, Literal, Sequence, TypedDict, List, Dict

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, JsonOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langgraph.graph import StateGraph, START, END

####### 2. API 키 로딩 #######
def load_api_keys(filepath="api_key.txt"):
    colab_path = '/content/drive/MyDrive/project_genai/api_key.txt'
    local_path = 'api_key.txt'

    if os.path.exists(colab_path):
        key_path = colab_path
    elif os.path.exists(local_path):
        key_path = local_path
    else:
        print("API Key 파일을 찾을 수 없습니다. OPENAI_API_KEY 환경 변수를 직접 설정하세요.")
        if not os.environ.get("OPENAI_API_KEY"):
             raise ValueError("OPENAI_API_KEY가 설정되지 않았습니다.")
        return

    with open(key_path, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

try:
    load_api_keys()
    if os.environ.get("OPENAI_API_KEY"):
        print("OpenAI API Key 로드 성공.")
    else:
        print("API Key 파일에서 키를 찾지 못했습니다. 환경 변수를 확인하세요.")
except Exception as e:
    print(f"API Key 로딩 중 오류 발생: {e}")


####### 3. 핵심 함수 및 Agent 로직 (팀과제 기반) #######

DEBUG_MODE = False  # ⚙️ True면 디버깅 로그 출력

# 1) 파일 입력
def extract_text_from_file(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        doc = fitz.open(file_path)
        text = "\n".join(page.get_text() for page in doc)
        doc.close()
        return text
    elif ext == ".docx":
        doc = Document(file_path)
        return "\n".join(p.text for p in doc.paragraphs if p.text.strip())
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 허용됩니다.")

# 2) State 선언 (고도화 버전)
class InterviewState(TypedDict):
    resume_text: str
    target_company:str
    resume_summary: str
    resume_keywords: List[str]
    target_company_keywords: List[str]
    question_strategy: Dict[str, Dict]
    current_question: str
    current_answer: str
    current_strategy: str
    conversation: List[Dict[str, str]]
    evaluation : List[Dict[str, str]]
    next_step : str
    reflection_status: str
    reflection_detail: Dict[str, str]
    last_evaluation: Dict[str, str]
    generate_count: str
    final_summary: str
    question_queue: List[str]
    remaining_topics: List[str]
    topic_map: Dict[str, List]

# 3) Resume 분석 (고도화 버전)
def analyze_resume(state: InterviewState) -> InterviewState:
    resume_text = state.get('resume_text')
    target_company = state.get('target_company')

    if not resume_text or resume_text.strip() == "":
        return {**state, "next_step": "end", "final_summary": "오류: 이력서 텍스트가 없습니다."}
    if not target_company or target_company.strip() == "":
        return {**state, "next_step": "end", "final_summary": "오류: 채용공고 텍스트가 없습니다."}

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    keywords_prompt = ChatPromptTemplate.from_template("""
    당신은 채용담당자입니다. 이력서와 채용공고를 분석하여 핵심 요약과 키워드를 JSON으로 반환하세요.
    [이력서]
    {resume_text}
    [채용공고]
    {target_company}
    [출력 형식]
    {{
      "resume_summary": "이력서 핵심 요약 (3~4문장)",
      "resume_keywords": ["키워드1", ..., "키워드10"],
      "target_company_keywords": ["키워드1", ..., "키워드10"]
    }}
    """)
    keywords_chain = keywords_prompt | llm | JsonOutputParser()
    try:
        result = keywords_chain.invoke({
            "resume_text": resume_text,
            "target_company": target_company
        })
    except Exception as e:
        return {**state, "next_step": "end", "final_summary": f"오류: 이력서 분석 실패 (LLM 오류) - {e}"}

    return {
        **state,
        "resume_summary": result.get("resume_summary", ""),
        "resume_keywords": result.get("resume_keywords", []),
        "target_company_keywords": result.get("target_company_keywords", []),
        "next_step": "generate_question_strategy"
    }

# 4) 질문 전략 수립 (고도화 버전)
def generate_question_strategy(state: InterviewState) -> InterviewState:
    resume_summary = state.get('resume_summary')
    resume_keywords = state.get('resume_keywords')
    target_company_keywords = state.get('target_company_keywords')

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    parser = JsonOutputParser()

    prompt = ChatPromptTemplate.from_template("""
    당신은 기업 면접관입니다. 이력서 요약과 키워드를 참고하여 면접 질문 전략을 수립하세요.
    [이력서 요약] {resume_summary}
    [지원자 키워드] {resume_keywords}
    [회사 키워드] {target_company_keywords}

    면접 질문 전략은 다음 3가지 분야로 구성합니다:
    1. 경력과 경험 (지원자의 과거 경험과 직무 역량 검증)
    2. 커뮤니케이션 능력 (팀워크, 협업, 갈등 해결 능력 검증)
    3. 문제해결력 (돌발 상황, 기술적 난관 해결 방식 검증)

    각 분야별로 아래 JSON 형식으로 작성하세요:
    {{
      "strategy_summary": "전체 전략 요약 (2~3문장)",
      "categories": [
        {{
          "category": "경력과 경험",
          "question_direction": "면접관이 어떤 관점으로 질문을 이끌어갈지에 대한 설명",
          "example_question": ["예시 질문 1", "예시 질문 2"]
        }},
        {{
          "category": "커뮤니케이션 능력",
          "question_direction": "면접관이 확인하려는 의도와 평가 포인트",
          "example_question": ["예시 질문 1", "예시 질문 2"]
        }},
        {{
          "category": "문제해결력",
          "question_direction": "지원자의 판단력, 책임감 등을 평가하기 위한 방향",
          "example_question": ["예시 질문 1", "예시 질문 2"]
        }}
      ]
    }}
    """)
    chain = prompt | llm | parser
    try:
        result = chain.invoke({
            "resume_summary": resume_summary,
            "resume_keywords": resume_keywords,
            "target_company_keywords": target_company_keywords
        })
    except Exception as e:
        return {**state, "next_step": "end", "final_summary": f"오류: 질문 전략 수립 실패 (LLM 오류) - {e}"}

    strategy_dict = {
        "summary": result.get("strategy_summary", ""),
        "categories": result.get("categories", [])
    }

    return {
        **state,
        "question_strategy": strategy_dict,
        "next_step": "initialize_queues"
    }

# 5) 답변 입력 (고도화 버전)
def update_current_answer(state: InterviewState) -> InterviewState:
    return {
        **state,
        "next_step": "evaluate_answer"
    }

# 6) 답변 평가 (고도화 버전)
def evaluate_answer(state: dict) -> dict:
    current_question = state.get("current_question")
    current_answer = state.get("current_answer")
    conversation = state.get("conversation", [])
    evaluation = state.get("evaluation", [])

    if not current_question or not current_answer:
        return {
            **state,
            "next_step": "summarize",
            "final_summary": "오류: 지원자가 응답하지 않아 면접을 종료합니다."
        }

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    parser = JsonOutputParser()
    prompt = ChatPromptTemplate.from_template(
        """당신은 AI 면접관의 답변 평가 전문가입니다.
        질문과 답변을 바탕으로 아래 기준을 0~100점으로 채점하고, 가중치를 적용한 총점을 계산하세요.
        [평가 기준 및 가중치]
        - 연관성 (0.30), 구체성 (0.20), 영향/성과 (0.20), 역할·주도성 (0.20), 커뮤니케이션(명료성) (0.10)
        [출력 형식(JSON)]
        {{
          "기준": {{
            "연관성": {{ "점수": <0~100 정수>, "근거": "<문장>" }},
            "구체성": {{ "점수": <0~100 정수>, "근거": "<문장>" }},
            "영향/성과": {{ "점수": <0~100 정수>, "근거": "<문장>" }},
            "역할·주도성": {{ "점수": <0~100 정수>, "근거": "<문장>" }},
            "커뮤니케이션(명료성)": {{ "점수": <0~100 정수>, "근거": "<문장>" }}
          }},
          "가중합_점수": <0~100 정수>,
          "등급": "<상|중|하>",
          "핵심피드백": "<1~2문장 요점>",
          "개선가이드": ["<실행 팁 1>", "<실행 팁 2>"]
        }}
        [등급 기준] (81-100: 상), (51-80: 중), (0-50: 하)
        ---
        [질문]: {question}
        [답변]: {answer}
        """
    )
    chain = prompt | llm | parser

    try:
        eval_result = chain.invoke({
            "question": current_question,
            "answer": current_answer,
        })
    except Exception as e:
        if DEBUG_MODE: print(f"❌ [DEBUG] evaluate_answer 오류: {e}")
        eval_result = {}

    conversation.append({"question": current_question, "answer": current_answer})
    evaluation.append({
        "question": current_question,
        "answer": current_answer,
        "evaluation": eval_result,
    })

    return {
        **state,
        "conversation": conversation,
        "evaluation": evaluation,
        "last_evaluation": eval_result,
        "next_step": "reflect",
    }

# 7) 평가 검증 (Reflection) (고도화 버전)
def reflect(state: InterviewState) -> InterviewState:
    current_question = state.get("current_question", "")
    current_answer = state.get("current_answer", "")
    last_evaluation = state.get("last_evaluation", None)

    if not last_evaluation:
        return {
            **state,
            "reflection_status": "재평가 필요",
            "reflection_detail": { "사유": "1차 평가(E) 없음" },
            "next_step": "re_evaluate_answer",
        }

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    parser = JsonOutputParser()
    prompt = ChatPromptTemplate.from_template(
        """당신은 "감수관"입니다. Q에 대한 A를 평가한 E가 적절했는지 평가합니다.
        [출력(JSON)]
        {{
          "평가_적절성_점수": (1~100),
          "판정": "적절 | 부분 부적절 | 부적절",
          "핵심사유": "",
          "과대과소_판정": "과대 | 과소 | 중립",
          "권장_수정": {{ "평가_총점": (1~100), "평가_코멘트": "E가 따를 보완 지시" }}
        }}
        --- Q(질문) ---
        {question}
        --- A(답변) ---
        {answer}
        --- E(LLM의 평가, JSON) ---
        {evaluation}
        """
    )
    chain = prompt | llm | parser
    eval_str = json.dumps(last_evaluation, ensure_ascii=False)

    try:
        result = chain.invoke({
            "question": current_question,
            "answer": current_answer,
            "evaluation": eval_str
        })
        adequacy = int(result.get("평가_적절성_점수", 0))
        verdict = (result.get("판정") or "").strip()
        is_ok = (adequacy >= 70) or (verdict == "적절")
    except Exception as e:
        if DEBUG_MODE: print(f"❌ [DEBUG] reflect 오류: {e}")
        is_ok = False
        result = {"사유": f"Reflection 오류: {e}"}


    if not is_ok:
        return {
            **state,
            "reflection_status": "재평가 필요",
            "reflection_detail": result,
            "next_step": "re_evaluate_answer"
        }

    return {
        **state,
        "reflection_status": "정상",
        "reflection_detail": result,
        "next_step": "decide_next_step"
    }

# 8) 답변 재평가 (고도화 버전)
def re_evaluate_answer(state: InterviewState) -> InterviewState:
    current_question = state.get("current_question")
    current_answer = state.get("current_answer")
    evaluation = state.get("evaluation", [])
    reflection_detail = state.get("reflection_detail", {})

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    parser = JsonOutputParser()
    prompt = ChatPromptTemplate.from_template(
        """당신은 AI 면접관의 재평가자입니다.
        질문과 답변을 바탕으로 아래 기준을 0~100점으로 채점하고, 가중치를 적용한 총점을 계산하세요.
        단, 아래의 감수 리포트를 반영하세요.

        [평가 기준 및 가중치]
        - 연관성 (0.30), 구체성 (0.20), 영향/성과 (0.20), 역할·주도성 (0.20), 커뮤니케이션(명료성) (0.10)

        [감수 리포트(반영할 지적사항)]
        {reflection_detail}

        [출력 형식(JSON)]
        {{
          "기준": {{ ... (evaluate_answer와 동일) ... }},
          "가중합_점수": <0~100 정수>,
          "등급": "<상|중|하>",
          "핵심피드백": "<1~2문장 요점>",
          "개선가이드": ["<실행 팁 1>", "<실행 팁 2>"]
        }}

        [등급 기준] (81-100: 상), (51-80: 중), (0-50: 하)
        ---
        [질문]: {question}
        [답변]: {answer}
        """
    )
    chain = prompt | llm | parser

    try:
        eval_result = chain.invoke({
            "question": current_question,
            "answer": current_answer,
            "reflection_detail": json.dumps(reflection_detail, ensure_ascii=False)
        })
    except Exception as e:
        if DEBUG_MODE: print(f"❌ [DEBUG] re_evaluate_answer 오류: {e}")
        eval_result = {}

    if evaluation and evaluation[-1].get("question") == current_question:
        evaluation[-1]["evaluation"] = eval_result
    else:
        evaluation.append({
            "question": current_question,
            "answer": current_answer,
            "evaluation": eval_result,
        })

    return {
        **state,
        "evaluation": evaluation,
        "last_evaluation": eval_result,
        "next_step": "decide_next_step"
    }

# 9) 다음 질문 큐에서 가져오기 (고도화 버전)
def get_next_question_from_queue(state: InterviewState) -> InterviewState:
    question_queue = state.get("question_queue", [])
    if not question_queue:
        return {**state, "next_step": "next_topic"}

    next_question = question_queue.pop(0)

    return {
        **state,
        "current_question": next_question,
        "question_queue": question_queue,
        "current_answer": "",
        "generate_count": "0",
        "next_step": "END"
    }

# 10) 다음 주제 질문 가져오기 (고도화 버전)
def next_topic_question(state: InterviewState) -> InterviewState:
    remaining_topics = state.get("remaining_topics", [])
    topic_map = state.get("topic_map", {})

    if not remaining_topics:
        return {**state, "next_step": "end"}

    current_topic = remaining_topics.pop(0)
    question_queue = list(topic_map.get(current_topic, [])) # ⭐️ list()로 복사

    if not question_queue:
        return {
            **state,
            "current_strategy": current_topic,
            "remaining_topics": remaining_topics,
            "question_queue": [],
            "generate_count": "0",
            "next_step": "generate"
        }

    selected_question = question_queue.pop(0)

    return {
        **state,
        "current_question": selected_question,
        "current_strategy": current_topic,
        "remaining_topics": remaining_topics,
        "question_queue": question_queue,
        "current_answer": "",
        "generate_count": "0",
        "next_step": "END"
    }

# 11) 인터뷰 진행 검토 (고도화 버전 - 규칙 기반)
def decide_next_step(state: InterviewState) -> InterviewState:
    conversation_count = len(state.get("conversation", []))
    last_evaluation = state.get("last_evaluation", {})
    question_queue = state.get("question_queue", [])
    remaining_topics = state.get("remaining_topics", [])

    if conversation_count >= 6:
        return {**state, "next_step": "end"}

    last_grade = last_evaluation.get("등급", "중")
    if last_grade == "하":
        return {**state, "next_step": "generate"}

    if question_queue:
        return {**state, "next_step": "next_question"}

    if not question_queue and remaining_topics:
        return {**state, "next_step": "next_topic"}

    return {**state, "next_step": "end"}


# 12) 질문 생성 (심화질문) (고도화 버전 - Chroma DB 연동)
def generate_question(state: InterviewState) -> InterviewState:
    resume_summary = state.get("resume_summary", "")
    resume_keywords = state.get("resume_keywords", [])
    current_strategy = state.get("current_strategy", "")
    conversation = state.get("conversation", [])
    generate_count = int(state.get("generate_count", 0))

    history_str = ""
    for i, conv in enumerate(conversation):
        history_str += f"\nQ{i+1}: {conv['question']}\nA{i+1}: {conv['answer']}\n"
    if not history_str: history_str = "아직 면접이 시작되지 않았습니다."

    if generate_count > 1:
        return {
            **state,
            "next_step": "next_topic",
            "generate_count": "0"
        }

    depth_comment = "이전 답변을 바탕으로, 지원자의 역량을 더 깊이 검증할 수 있는 심화 질문을 생성하세요."

    # Chroma Vector DB 연동
    try:
        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        db_path = "./chroma_db"
        if not os.path.exists(db_path):
            os.makedirs(db_path)
            db = Chroma.from_texts(texts=["샘플 질문 1"], embedding=embeddings, persist_directory=db_path)
        else:
            db = Chroma(persist_directory=db_path, embedding_function=embeddings)

        query_text = f"{resume_summary}\n{history_str}"
        similar_docs = db.similarity_search(query_text, k=3)
        similar_questions = "\n".join([f"- {doc.page_content}" for doc in similar_docs])
    except Exception as e:
        similar_questions = "유사 질문 정보를 불러올 수 없습니다."
        if DEBUG_MODE: print(f"[Chroma 검색 오류] {e}")

    prompt = ChatPromptTemplate.from_template(
        """당신은 전문 AI 면접관입니다.
        아래 정보를 참고하여 지원자에게 **새로운 심층 질문**을 하나만 생성하세요.
        [참고: 기존 면접 질문]
        {similar_questions}
        [규칙]
        1. 위 유사 질문들과 **겹치지 않는 새로운 질문**을 만드세요.
        2. 지원자의 이전 답변을 바탕으로 사고력, 문제해결력, 가치관을 탐색하세요.
        3. 질문은 한 문장으로, 명확하고 간결하게 작성하세요.
        4. {depth_comment}
        ---
        [지원자 이력서 요약] {summary}
        [현재 면접 주제] {strategy}
        [지금까지의 면접 기록] {history}
        ---
        [다음 심층 질문 (오직 한 문장)]:
        """
    )

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
    chain = prompt | llm | StrOutputParser()

    response = chain.invoke({
        "summary": resume_summary,
        "strategy": current_strategy,
        "history": history_str,
        "depth_comment": depth_comment,
        "similar_questions": similar_questions
    })

    return {
        **state,
        "current_question": response.strip(),
        "current_answer": "",
        "generate_count": str(generate_count + 1),
        "next_step": "END"
    }

# 13) 인터뷰 피드백 보고서 (⭐️⭐️⭐️ 버그 수정 ⭐️⭐️⭐️)
def summarize_interview(state: InterviewState) -> InterviewState:
    report_lines = []
    report_lines.append("== AI 면접 최종 피드백 보고서 ==")

    report_lines.append("\n[이력서 요약]")
    report_lines.append(state.get("resume_summary", "요약 정보 없음"))

    conversation = state.get("conversation", [])
    evaluation = state.get("evaluation", [])

    if not conversation or not evaluation:
        report_lines.append("\n면접 기록이 없습니다.")
        return {**state, "final_summary": "\n".join(report_lines), "next_step": "END"}

    report_lines.append("\n== 면접 상세 내역 ==")
    for i, (conv, eval_item) in enumerate(zip(conversation, evaluation), 1):
        eval_data = eval_item.get("evaluation", {})
        criteria = eval_data.get("기준", {})
        report_lines.append(f"\n[질문 {i}] Q: {conv.get('question', 'N/A')}")
        report_lines.append(f"A: {conv.get('answer', 'N/A')}")
        report_lines.append(f"  [평가] 총점: {eval_data.get('가중합_점수', 'N/A')} (등급: {eval_data.get('등급', 'N/A')})")
        report_lines.append(f"  - 피드백: {eval_data.get('핵심피드백', 'N/A')}")
        report_lines.append("-" * 40)

    # 3. 종합 평가 (LLM)
    report_lines.append("\n== 종합 평가 ==")
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # (⭐️⭐️⭐️ 핵심 수정 ⭐️⭐️⭐️)
    # report_lines.index()는 \n이 포함된 문자열을 찾아야 합니다.
    try:
        start_idx = report_lines.index("\n== 면접 상세 내역 ==")
        interview_summary_text = "\n".join(report_lines[start_idx:])
    except ValueError:
        interview_summary_text = "\n".join(report_lines) # 못찾으면 전체 전송

    comprehensive_prompt = ChatPromptTemplate.from_template(
        """당신은 전문 채용 컨설턴트입니다.
        다음 면접 기록을 바탕으로 **구체적이고 실질적인** 종합 평가를 작성하세요.
        **면접 기록:**
        {interview_text}
        **다음 형식으로 구체적으로 작성하세요:**
        1. 전체 인상
        2. 강점 (2-3가지)
        3. 약점 (2-3가지)
        4. 개선 방안 (2-3가지)
        """
    )
    comprehensive_chain = comprehensive_prompt | llm | StrOutputParser()
    try:
        comprehensive_feedback = comprehensive_chain.invoke({
            "interview_text": interview_summary_text
        })
        report_lines.append(comprehensive_feedback)
    except Exception as e:
        report_lines.append(f"종합 평가 생성 중 오류 발생: {e}")

    # 4. 합격 가능성 분석 (규칙 기반)
    report_lines.append("\n== 모의 합격 가능성 분석 ==")
    total_score, max_score, good_count, poor_count = 0, 0, 0, 0

    for eval_item in evaluation:
        eval_dict = eval_item.get("evaluation", {})
        grade = eval_dict.get("등급", "하")
        if grade == "상": total_score += 3; good_count += 1
        elif grade == "중": total_score += 2; good_count += 1
        else: total_score += 1; poor_count += 1
        max_score += 3

    base_pass_rate = (total_score / max_score * 100) if max_score > 0 else 0
    count_bonus = min(len(conversation) * 5, 20)
    final_pass_rate = min(base_pass_rate + count_bonus, 95)

    if final_pass_rate >= 85: possibility, probability, comment = "매우 높음", "85-95%", "우수한 답변을 하였고, 충분히 검증되었습니다."
    elif final_pass_rate >= 70: possibility, probability, comment = "높음", "70-85%", "전반적으로 좋은 평가를 받았습니다."
    elif final_pass_rate >= 55: possibility, probability, comment = "보통", "50-70%", "기본은 갖췄으나, 보완이 필요합니다."
    else: possibility, probability, comment = "낮음", "50% 미만", "여러 영역에서 개선이 필요합니다."

    report_lines.append(f"[합격 가능성] {possibility} ({probability})")
    report_lines.append(f"(기본 {base_pass_rate:.1f}% + 면접 보너스 {count_bonus}% = {final_pass_rate:.1f}%)")
    report_lines.append(f"{comment}")
    report_lines.append("\n== 면접 종료 ==")

    final_report_str = "\n".join(report_lines)

    return {
        **state,
        "final_summary": final_report_str,
        "next_step": "END"
    }


####### 4. LangGraph 정의 (Gradio 호환) #######

builder = StateGraph(InterviewState)

builder.add_node("evaluate", evaluate_answer)
builder.add_node("reflection", reflect)
builder.add_node("re_evaluate_answer", re_evaluate_answer)
builder.add_node("decide_next", decide_next_step)
builder.add_node("generate_question", generate_question)
builder.add_node("get_next_question_from_queue", get_next_question_from_queue)
builder.add_node("next_topic_question", next_topic_question)
builder.add_node("summarize", summarize_interview)

builder.set_entry_point("evaluate")
builder.add_edge("evaluate", "reflection")

builder.add_conditional_edges(
    "reflection",
    lambda s: s.get("reflection_status"),
    { "정상": "decide_next", "재평가 필요": "re_evaluate_answer" }
)
builder.add_edge("re_evaluate_answer", "decide_next")

builder.add_conditional_edges(
    "decide_next",
    lambda s: s.get("next_step"),
    {
        "generate": "generate_question",
        "next_question": "get_next_question_from_queue",
        "next_topic": "next_topic_question",
        "end": "summarize"
    }
)

# ⭐️ (수정) generate_question 또는 next_topic_question이
# 다음 주제(next_topic)로 가거나 끝(end)나야 할 때
builder.add_conditional_edges(
    "generate_question",
    lambda s: s.get("next_step"),
    {
        "next_topic": "next_topic_question", # 심화질문 2회 초과 시 다음 주제로
        "END": END # 일반적인 심화질문 생성 시 Gradio 대기
    }
)

builder.add_conditional_edges(
    "next_topic_question",
    lambda s: s.get("next_step"),
    {
        "generate": "generate_question",
        "END": END,
        "end": "summarize" # ⭐️ 남은 주제가 없을 때
    }
)

builder.add_edge("get_next_question_from_queue", END)
builder.add_edge("summarize", END)

graph = builder.compile()
print("✅ [PROD] 면접 Agent 그래프가 성공적으로 컴파일되었습니다!")


####### 5. Gradio 인터페이스 #######

# 세션 상태 초기화 함수
def initialize_state():
    return {
        "state": {
            "resume_text": "", "target_company": "", "resume_summary": "",
            "resume_keywords": [], "target_company_keywords": [],
            "question_strategy": {}, "current_question": "", "current_answer": "",
            "current_strategy": "", "conversation": [], "evaluation": [],
            "next_step": "", "generate_count": "0", "reflection_status": "",
            "reflection_detail": {}, "last_evaluation": {}, "final_summary": "",
            "question_queue": [], "remaining_topics": [], "topic_map": {}
        },
        "interview_started": False,
        "interview_ended": False,
        "chat_history": []
    }

# 파일 업로드 후 인터뷰 초기화 (Gradio용으로 수정)
def upload_and_initialize(resume_file, company_file, session_state):
    if resume_file is None or company_file is None:
        return session_state, [["🤖 AI 면접관", "이력서와 채용공고 파일을 모두 업로드해주세요."]], gr.update(visible=True), gr.update(visible=False)

    try:
        state = initialize_state()["state"]
        state["resume_text"] = extract_text_from_file(resume_file.name)
        state["target_company"] = extract_text_from_file(company_file.name)

        state = analyze_resume(state)
        if state["next_step"] == "end":
             return session_state, [["🤖 AI 면접관", state["final_summary"]]], gr.update(visible=True), gr.update(visible=False)

        state = generate_question_strategy(state)

        categories = state["question_strategy"].get("categories", [])
        topic_map = {cat.get("category"): cat.get("example_question", []) for cat in categories if cat.get("category")}
        remaining_topics = [cat.get("category") for cat in categories if cat.get("category")]

        if not remaining_topics:
             return session_state, [["🤖 AI 면접관", "질문 전략 수립에 실패했습니다."]], gr.update(visible=True), gr.update(visible=False)

        first_topic = remaining_topics.pop(0)
        question_queue = list(topic_map.get(first_topic, [])) # ⭐️ list()로 복사

        if not question_queue:
            first_question = f"{first_topic}에 대해 자유롭게 말씀해 주세요."
        else:
            first_question = question_queue.pop(0)

        state["current_question"] = first_question
        state["current_strategy"] = first_topic
        state["question_queue"] = question_queue
        state["remaining_topics"] = remaining_topics
        state["topic_map"] = topic_map
        state["next_step"] = "answer"

        session_state["state"] = state
        session_state["interview_started"] = True
        session_state["chat_history"] = [["🤖 AI 면접관", "준비가 완료되었습니다. 첫 번째 질문입니다." + "\n\n" + first_question]]

        return session_state, session_state["chat_history"], gr.update(visible=False), gr.update(visible=True)

    except Exception as e:
        return session_state, [["🤖 AI 면접관", f"오류 발생: {e}"]], gr.update(visible=True), gr.update(visible=False)


# 답변 처리 및 다음 질문 생성 (⭐️⭐️⭐️ 버그 수정 ⭐️⭐️⭐️)
def chat_interview(user_input, session_state):
    if not session_state["interview_started"] or session_state["interview_ended"]:
        return session_state, session_state["chat_history"], gr.update(value="")

    if not user_input or user_input.strip() == "":
        session_state["chat_history"].append(["🤖 AI 면접관", "답변이 비어있습니다. 답변을 입력해주세요."])
        return session_state, session_state["chat_history"], gr.update(value="")

    # (1) 사용자 답변 저장
    session_state["chat_history"].append(["🙋‍♂️ 지원자", user_input])

    # (2) State에 답변 업데이트
    state = session_state["state"]
    state["current_answer"] = user_input.strip()
    state["next_step"] = "evaluate"

    # (3) Agent 그래프 실행 (evaluate ~ END)
    try:
        if DEBUG_MODE: print(f"\n--- 🚀 graph.invoke() 직전 (Next Step: {state['next_step']}) ---")

        # ⭐️⭐️⭐️ 핵심 수정 ⭐️⭐️⭐️
        # state를 받아 state를 반환하는 'graph.invoke()'는
        # state 딕셔너리 *전체*를 입력으로 받아야 합니다.
        state = graph.invoke(state)

        if DEBUG_MODE: print(f"\n--- 🏁 graph.invoke() 직후 (Next Step: {state['next_step']}) ---")

    except Exception as e:
        if DEBUG_MODE: print(f"❌ [DEBUG] graph.invoke 오류: {e}")
        state["next_step"] = "end"
        state["final_summary"] = f"면접 중 오류가 발생하여 종료합니다. (오류: {e})"
        # 오류 발생 시에도 세션 상태를 업데이트해야 합니다.
        session_state["state"] = state


    # (4) Gradio 세션 상태 업데이트
    session_state["state"] = state

    # (5) 종료 여부 판단 (final_summary가 채워졌는지 확인)
    if state.get("final_summary"):
        session_state["interview_ended"] = True
        final_summary = state.get("final_summary")
        session_state["chat_history"].append(["🤖 AI 면접관", final_summary])

        return session_state, session_state["chat_history"], gr.update(value="면접 종료됨", interactive=False)

    # (6) 다음 질문 진행
    # ⭐️⭐️⭐️ 핵심 수정 ⭐️⭐️⭐️
    # state['next_step']이 'next_topic' 또는 'generate'일 수 있습니다. (심화질문 2회 초과 / 큐가 빈 경우)
    # 이 경우, 그래프를 다시 호출하여 다음 질문을 가져와야 합니다.
    elif state["next_step"] in ["next_topic", "generate"]:
        if DEBUG_MODE: print(f"    -> 다음 턴 자동 실행: {state['next_step']}")
        # state['current_answer']를 비워야 evaluate에서 막히지 않습니다.
        state["current_answer"] = ""
        # chat_interview를 재귀적으로 호출하여 다음 질문을 바로 받아옵니다.
        # (user_input은 "Auto-Trigger"와 같은 임시 값으로 채워도 무방합니다)
        return chat_interview(" ", session_state)

    else:
        # (next_step == "END" 상태)
        # 다음 질문 표시 (generate, next_question, next_topic에서 생성됨)
        next_question = state["current_question"]
        session_state["chat_history"].append(["🤖 AI 면접관", next_question])
        return session_state, session_state["chat_history"], gr.update(value="")

# (Gradio) 재시작 함수
def restart_interview():
    return initialize_state(), [], gr.update(visible=True), gr.update(visible=False), gr.update(value="", interactive=True)

# Gradio 인터페이스 구성 (2개 파일 입력으로 수정)
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    session_state = gr.State(initialize_state)

    gr.Markdown("# 🤖 AI 면접관 (고도화 v2.5)")
    gr.Markdown("이력서와 지원하는 포지션의 채용공고(JD)를 업로드하고 인터뷰를 시작하세요.")

    with gr.Group(visible=True) as upload_group:
        with gr.Row():
            resume_file = gr.File(label="1. 이력서 업로드 (PDF 또는 DOCX)")
            company_file = gr.File(label="2. 채용공고 업로드 (PDF 또는 DOCX)")
        upload_btn = gr.Button("인터뷰 시작", variant="primary")

    with gr.Group(visible=False) as chat_group:
        chatbot = gr.Chatbot(label="AI 면접관", height=500, show_copy_button=True, bubble_full_width=True)
        user_input = gr.Textbox(show_label=False, placeholder="답변을 입력하고 Enter를 누르세요.")
        restart_btn = gr.Button("처음부터 다시 시작")

    upload_btn.click(
        upload_and_initialize,
        inputs=[resume_file, company_file, session_state],
        outputs=[session_state, chatbot, upload_group, chat_group]
    )

    user_input.submit(
        chat_interview,
        inputs=[user_input, session_state],
        outputs=[session_state, chatbot, user_input]
    )

    restart_btn.click(
        restart_interview,
        inputs=[],
        outputs=[session_state, chatbot, upload_group, chat_group, user_input]
    )


# 실행
if __name__ == "__main__":
    demo.launch(share=True, debug=True)

Overwriting app.py


## **3. 실행**

In [17]:
!python app.py

OpenAI API Key 로드 성공.
✅ [PROD] 면접 Agent 그래프가 성공적으로 컴파일되었습니다!
/content/app.py:870: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="AI 면접관", height=500, show_copy_button=True, bubble_full_width=True)
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://6b4a008533fb9544b2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
/content/app.py:511: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the `langcha